### 1. Import Library

In [1]:
import requests
from bs4 import BeautifulSoup
from fpdf import FPDF
import pandas as pd

### 2.Crawl_data_q_a

#### 2.1.Find Elements

In [2]:
# Hàm để cào dữ liệu từ một trang
def crawl_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    articles = soup.find_all('div', class_='box-item-content')
    articles_data = []
    
    for _, article in enumerate(articles, start=1):
        # Lấy thông tin tác giả và thời gian
        author_info = article.find('div', class_='box-item-top')
        name = author_info.find('span', class_='author').get_text(strip=True)
        time = author_info.find('span', class_='time').get_text(strip=True)

        # Lấy tiêu đề câu hỏi
        question_element = article.find('a', class_='question-title')
        question = question_element.get_text(strip=True)
        
        # Lấy tình huống
        summary = article.find('div', class_='question-sapo').get_text(strip=True)
        
        # Lấy liên kết chi tiết
        detail_link = article.find('a', class_='box-viewmore')['href']
        detail_url = f"https://chinhsachonline.chinhphu.vn{detail_link}"
        
        # Gửi yêu cầu HTTP để lấy nội dung trang chi tiết
        detail_response = requests.get(detail_url)
        detail_soup = BeautifulSoup(detail_response.content, 'html.parser')
        
        # Lấy câu trả lời từ trang chi tiết
        answer_div = detail_soup.find('div', class_='detail__rcontent')
        answer = answer_div.get_text(strip=True) if answer_div else 'No answer available'
        
        # Thêm thông tin bài viết vào danh sách
        articles_data.append({
            'Name': name,
            'Time': time,
            'Question': question,
            'Situation': summary,
            'Answer': answer
        })
    
    return articles_data


#### 2.2.Execute Function

In [3]:
articles_data = []

for page_num in range(1, 301):  # Thay đổi phạm vi để bao gồm số trang bạn cần
    page_url = f"https://chinhsachonline.chinhphu.vn/giao-thong-xay-dung-tai-nguyen-moi-truong/t1/13/trang-{page_num}.htm"
    articles_data.extend(crawl_page(page_url))
    

#### 2.3 Export csv

In [4]:
# Tạo DataFrame từ danh sách dữ liệu
df_laodong_tienthuong = pd.DataFrame(articles_data)
df_laodong_tienthuong.head(5)

,Name,Time,Question,Situation,Answer
0,Tạ Hữu Mạnh,08:14 09/08/2024,Đăng ký cấp quyền sử dụng đất lần đầu cần giấy...,Tại Mục 1 Đăng ký quyền sử dụng đất lần đầu đố...,Bộ Tài nguyên và Môi trường trả lời vấn đề này...
1,Nguyễn Thị Thúy,14:18 08/08/2024,Có thể nghiệm thu từng phần công trình xây dựng,Công trình A được phê duyệt kế hoạch lựa chọn ...,Bộ Xây dựng trả lời vấn đề này như sau:Theo qu...
2,Yên Giang,08:08 08/08/2024,"Dự án trên 15 tỷ đồng, xác định chi phí tư vấn...",Thông tư số 12/2021/TT-BXD ngày 31/8/2021 của ...,Bộ Xây dựng trả lời vấn đề này như sau:Việc ph...
3,Ứng Quốc Tiến,09:34 07/08/2024,"Sổ đỏ có sai sót, điều chỉnh thế nào?",Tôi được cấp Giấy chứng nhận quyền sử dụng đất...,Bộ Tài nguyên và Môi trường trả lời vấn đề này...
4,Nguyễn Thị Khuyên,09:05 06/08/2024,"Thế nào là cho thuê bất động sản quy mô nhỏ, k...","Tôi có một doanh nghiệp thành lập năm 2018, nh...",Bộ Xây dựng trả lời vấn đề này như sau:Tại Kho...


In [5]:

# Lưu DataFrame vào file CSV
csv_file = '../data/giaothong_xaydung_tainguyen_moitruong_data.csv'
df_laodong_tienthuong.to_csv(csv_file, encoding='utf-8',index_label='Index')

print(f"Successfully saved data to {csv_file}")

Successfully saved data to ../data/giaothong_xaydung_tainguyen_moitruong_data.csv


In [6]:
df_laodong_tienthuong=pd.read_csv("../data/giaothong_xaydung_tainguyen_moitruong_data.csv",encoding='utf-8')

In [7]:
df_laodong_tienthuong.head(5)

,Index,Name,Time,Question,Situation,Answer
0,0,Tạ Hữu Mạnh,08:14 09/08/2024,Đăng ký cấp quyền sử dụng đất lần đầu cần giấy...,Tại Mục 1 Đăng ký quyền sử dụng đất lần đầu đố...,Bộ Tài nguyên và Môi trường trả lời vấn đề này...
1,1,Nguyễn Thị Thúy,14:18 08/08/2024,Có thể nghiệm thu từng phần công trình xây dựng,Công trình A được phê duyệt kế hoạch lựa chọn ...,Bộ Xây dựng trả lời vấn đề này như sau:Theo qu...
2,2,Yên Giang,08:08 08/08/2024,"Dự án trên 15 tỷ đồng, xác định chi phí tư vấn...",Thông tư số 12/2021/TT-BXD ngày 31/8/2021 của ...,Bộ Xây dựng trả lời vấn đề này như sau:Việc ph...
3,3,Ứng Quốc Tiến,09:34 07/08/2024,"Sổ đỏ có sai sót, điều chỉnh thế nào?",Tôi được cấp Giấy chứng nhận quyền sử dụng đất...,Bộ Tài nguyên và Môi trường trả lời vấn đề này...
4,4,Nguyễn Thị Khuyên,09:05 06/08/2024,"Thế nào là cho thuê bất động sản quy mô nhỏ, k...","Tôi có một doanh nghiệp thành lập năm 2018, nh...",Bộ Xây dựng trả lời vấn đề này như sau:Tại Kho...
